# Struktur Projeck:

In [ ]:
ALAT_WIFI/
├── static/
│   ├── css/
│   │   └── styles.css
│   └── js/
│       └── script.js
├── templates/
│   └── index.html
├── requirements.txt
├── app.py
└── wifi_tool.py

# Implementasi Kode dan analisa yang di gunakan untuk membuat program 

- `app.py`: Skrip utama untuk menjalankan aplikasi Flask. 

Analisa code : 

1. Struktur Data:
Struktur data utama yang digunakan dalam kode ini adalah dictionary, khususnya untuk menangani permintaan dan respons JSON.
Contohnya, 'request.json' adalah dictionary yang berisi data yang dikirim dalam permintaan POST, dan jsonify(result=result) mengembalikan dictionary sebagai respons JSON

2. Array Dimensi 1 & 2:
Tidak digunakan langsung dalam kode ini. Namun, jika fungsi-fungsi Alat_wifi mengembalikan list atau array, mereka akan ditangani sebagai bagian dari result.

3. Array Dimensi Banyak: 
Tidak digunakan langsung dalam kode ini. Konsep ini akan relevan jika fungsi-fungsi Alat_wifi mengembalikan array multi-dimensi. 
 
Kode ini terutama berfokus pada penanganan permintaan HTTP dan mengeksekusi fungsi-fungsi spesifik dari modul Alat_wifi berdasarkan aksi dalam permintaan. Kode ini menggunakan struktur data seperti dictionary untuk menangani data JSON dan Flask untuk routing dan penanganan permintaan. Konsep seperti tipe data dan struktur data digunakan secara implisit melalui dictionary dan manipulasi string dalam kode ini.

code :

In [ ]:
# app.py 
from flask import Flask, render_template, request, jsonify 
import Alat_wifi 

app = Flask(__name__) 

@app.route('/') 
def index(): 
    return render_template('index.html') 

@app.route('/execute', methods=['POST']) 
def execute(): 
    action = request.json['action'] 
    if action == "periksa_host_windows": 
        result = Alat_wifi.periksa_host_windows() 
    elif action == "list_ssids": 
        result = Alat_wifi.cmd_daftar_ssid_tersedia() 
    elif action == "list_profiles": 
        result = Alat_wifi.cmd_daftar_profil() 
    elif action == "list_blocked_aps": 
        result = Alat_wifi.cmd_daftar_ap_terblokir() 
    elif action == "show_interface": 
        result = Alat_wifi.cmd_tampilkan_antarmuka_saat_ini() 
    elif action == "full_report": 
        result = Alat_wifi.cmd_buat_laporan_lengkap() 
    elif action == "show_cleartext": 
        ap_name = request.json.get('ap_name') 
        result = Alat_wifi.cmd_tampilkan_password_cleartext(ap_name) 
    elif action == "dump_profiles": 
        result = Alat_wifi.cmd_dump_profil_terenkripsi() 
    else: 
        result = '[!] Pilihan Tidak Valid' 
    return jsonify(result=result) 

if __name__ == "__main__": 
    app.run(debug=True) 


- `Alat_wifi.py`: Skrip yang berisi fungsi-fungsi untuk menjalankan perintah CMD terkait Wi-Fi. 

Analisa code :

1. Tipe Data: Variabel-variabel yang digunakan seperti string (misalnya nama_ap), dan hasil yang dikembalikan oleh fungsi-fungsi adalah string. 

Kode ini fokus pada penggunaan fungsi-fungsi yang berinteraksi dengan sistem operasi Windows melalui perintah-perintah shell menggunakan modul subprocess. Implementasi ini lebih terkait dengan pemanggilan perintah eksternal untuk mengelola dan memeriksa konfigurasi jaringan WiFi pada sistem operasi Windows.

Code :

In [ ]:
import os 
import subprocess 
from subprocess import check_output 

def periksa_host_windows(): 
    if os.name != "nt": 
        return '[!] Silakan Jalankan Aplikasi Di Mesin Windows' 
    return '[+] Semua bagus....' 

def cmd_daftar_ssid_tersedia(): 
    return check_output("netsh wlan show networks", shell=True).decode('utf-8') 

def cmd_daftar_profil(): 
    return check_output("netsh wlan show profiles", shell=True).decode('utf-8') 

def cmd_daftar_ap_terblokir(): 
    return check_output("netsh wlan show filters", shell=True).decode('utf-8') 

def cmd_tampilkan_antarmuka_saat_ini(): 
    return check_output("netsh wlan show interfaces", shell=True).decode('utf-8') 

def cmd_buat_laporan_lengkap(): 
    return check_output("netsh wlan show all", shell=True).decode('utf-8') 

def cmd_tampilkan_password_cleartext(nama_ap): 
    if nama_ap: 
        return check_output(f"netsh wlan show profiles name=\"{nama_ap}\" key=clear", shell=True).decode('utf-8') 
    return "[!] Masukkan nama titik akses nirkabel yang benar." 

def cmd_buka_profil(): 
    return check_output("notepad profiles.txt", shell=True) 

def cmd_dump_profil_terenkripsi(): 
    return check_output("netsh wlan export profile folder=.", shell=True).decode('utf-8') 


# implementasi perintah netsh wlan yang di gunakan:

beberapa perintah yang digunakan beserta fungsinya:

1. **Menampilkan Daftar SSID**: 
   - Perintah: `netsh wlan show networks` 
   - Fungsi: Menampilkan daftar jaringan Wi-Fi (SSID) yang tersedia di sekitar perangkat. Berguna untuk mengetahui jaringan Wi-Fi yang dapat diakses dari lokasi saat ini. 
  
2. **Menampilkan Profil yang Tersimpan**: 
   - Perintah: `netsh wlan show profiles` 
   - Fungsi: Menampilkan daftar profil Wi-Fi yang telah disimpan di perangkat. Setiap kali Anda terhubung ke jaringan Wi-Fi baru, Windows akan menyimpan profil tersebut, termasuk informasi seperti nama jaringan (SSID) dan kata sandi. 
  
3. **Menampilkan Filter Wi-Fi**: 
   - Perintah: `netsh wlan show filters` 
   - Fungsi: Menampilkan filter yang telah diterapkan pada koneksi Wi-Fi, seperti daftar jaringan yang diperbolehkan atau diblokir. 
  
4. **Menampilkan Informasi Antarmuka Wi-Fi Saat Ini**: 
   - Perintah: `netsh wlan show interfaces` 
   - Fungsi: Menampilkan informasi detail mengenai antarmuka Wi-Fi yang sedang aktif, seperti nama antarmuka, status koneksi, kecepatan, dan kualitas sinyal. Berguna untuk memantau kondisi koneksi Wi-Fi dan melakukan diagnosa jika terjadi masalah. 
  
5. **Menampilkan Semua Informasi Wi-Fi**: 
   - Perintah: `netsh wlan show all` 
   - Fungsi: Menampilkan semua informasi terkait Wi-Fi, termasuk daftar jaringan, profil yang disimpan, dan informasi antarmuka. Perintah ini memberikan gambaran menyeluruh tentang status Wi-Fi di perangkat. 
  
6. **Menampilkan Detail Profil Tertentu**: 
   - Perintah: `netsh wlan show profiles name="{nama_ap}" key=clear` 
   - Fungsi: Menampilkan detail dari profil Wi-Fi tertentu, termasuk kata sandi dalam bentuk teks yang jelas (clear text). Sangat berguna ketika Anda perlu mengetahui atau memverifikasi informasi login jaringan Wi-Fi tertentu. 
  
7. **Mengekspor Profil Wi-Fi**: 
   - Perintah: `netsh wlan export profile folder=.""` 
   - Fungsi: Mengekspor profil Wi-Fi yang tersimpan ke folder yang ditentukan dalam bentuk file XML. File ini dapat digunakan untuk mengimpor profil tersebut ke perangkat lain atau sebagai backup. 
  
Dengan memanfaatkan perintah-perintah ini, pengguna dapat mengelola dan memantau koneksi Wi-Fi mereka secara efisien melalui CMD. Hal ini sangat membantu terutama bagi pengguna yang lebih nyaman bekerja dengan perintah teks atau saat melakukan troubleshooting masalah koneksi jaringan.

***analisis tentang konsep-konsep struktur data dan array yang digunakan dalam perintah `netsh wlan`**:

1. Menggunakan Konsep Struktur Data dan Array Apa yang Dipakai? 

- **Array**: Perintah-perintah `netsh wlan` kemungkinan besar menggunakan array untuk menyimpan daftar jaringan, profil, dan informasi lainnya. 
  - Contoh: Menyimpan daftar jaringan Wi-Fi yang ditemukan (`networks[]`) atau daftar profil yang tersimpan (`profiles[]`).
  
- **Struct**: Digunakan untuk menyimpan detail setiap jaringan atau profil, seperti SSID, tipe radio, jenis keamanan, dan kekuatan sinyal.

  - Contoh: Struktur data untuk menyimpan informasi jaringan bisa berupa:

In [ ]:
struct WifiNetwork { 
        char SSID[32]; 
        char RadioType[16]; 
        char SecurityType[16]; 
        int SignalStrength; 
    }; 
    struct WifiNetwork networks[MAX_NETWORKS]; 

2. Konsep Bilangan Apa yang Dipakai? 

- **Sistem Bilangan Desimal**: Digunakan untuk menampilkan informasi numerik seperti kekuatan sinyal (dalam desibel), jumlah jaringan yang ditemukan, kecepatan transmisi, dan alamat MAC. 
- **Sistem Bilangan Hexadesimal**: Digunakan dalam representasi alamat MAC. 
  
### 3. Konsep Array Dimensi Banyak Apa yang Dipakai? 
- **Array Dimensi Dua**: Digunakan untuk menyimpan data kompleks seperti daftar jaringan dan atributnya. Misalnya, `networks` sebagai array dari struct `WifiNetwork`. 
  - Contoh: 

In [ ]:
 struct WifiNetwork networks[MAX_NETWORKS]; 

4. Konsep Single Linked List Apa yang Dipakai? 

- **Single Linked List**: Tidak ada penggunaan eksplisit dari single linked list dalam perintah `netsh wlan`, tetapi mungkin digunakan di backend untuk mengelola daftar dinamis dari elemen-elemen seperti profil jaringan. 

10. Graph dan Matriks Penyajian Apa yang Dipakai? 

- **Graph dan Matriks Penyajian**: Perintah `netsh wlan` tidak secara eksplisit menggunakan graph atau matriks dalam penyajiannya, tetapi hubungan antara jaringan dan profil dapat dimodelkan sebagai graph. Misalnya, jaringan yang berbeda sebagai node dan profil yang menghubungkan mereka sebagai edge. 

# Rangkuman

Perintah `netsh wlan show networks`, `netsh wlan show profiles`, `netsh wlan show filters`, `netsh wlan show interfaces`, `netsh wlan show all`, `netsh wlan show profiles name="{nama_ap}" key=clear`, dan `netsh wlan export profile folder=.` menggunakan konsep array dan struct untuk mengelola dan menampilkan data jaringan. Sistem bilangan desimal dan hexadesimal digunakan untuk menampilkan informasi numerik. Konsep-konsep lain seperti linked list, stack, queue, pohon, dan graph tidak secara eksplisit diterapkan dalam konteks perintah-perintah ini. 

- `templates/index.html`: File HTML utama untuk antarmuka web. 

Analisa code:

bertanggung jawab untuk menampilkan antarmuka pengguna (user interface) menggunakan elemen-elemen seperti tombol '<button>', input teks '<input>', dan elemen-elemen lainnya untuk mengorganisir dan memformat konten. Implementasi logika atau manipulasi data berada di lapisan backend (misalnya, dengan Flask di server) atau mungkin dengan JavaScript yang diacu di dalam kode HTML seperti 'script.js'.

In [ ]:
<!DOCTYPE html> 
<html lang="id"> 
<head> 
    <meta charset="UTF-8"> 
    <meta name="viewport" content="width=device-width, initial-scale=1.0"> 
    <title>Alat Wifi</title> 
     <!--Bootstrap CSS--> 
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet"> 
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}"> 
</head> 
<body> 
    <div class="container mt-5"> 
        <h1 class="text-center">Alat Wifi</h1> 
        <div class="card mt-4"> 
            <div class="card-body"> 
                <div class="row"> 
                    <div class="col-md-6"> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('check_host_windows')">Periksa Host Windows</button> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('list_ssids')">Daftar SSID Tersedia</button> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('list_profiles')">Daftar Profil</button> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('list_blocked_aps')">Daftar AP Terblokir</button> 
                    </div> 
                    <div class="col-md-6"> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('show_interface')">Tampilkan Antarmuka Saat Ini</button> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('full_report')">Buat Laporan Lengkap</button> 
                        <input type="text" id="ap_name" class="form-control mb-2" placeholder="Masukkan Nama AksesPoin"> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('show_cleartext')">Tampilkan Password Cleartext</button> 
                        <button class="btn btn-primary btn-block mb-2" onclick="executeAction('dump_profiles')">Dump Profil Enkripsi</button> 
                    </div> 
                </div> 
            </div> 
        </div> 
        <pre id="output" class="mt-4 p-3 border rounded"></pre> 
    </div> 
    <!-- Bootstrap JS dan dependensi --> 
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script> 
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.5.4/dist/umd/popper.min.js"></script> 
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script> 
    <script src="{{ url_for('static', filename='js/script.js') }}"></script> 
</body> 
</html> 

- `static/css/styles.css`: File CSS untuk styling. 

Analisa code :

Code ini mengatur tampilan dari elemen-elemen HTML pada halaman web. Ini termasuk warna latar belakang (background-color), warna teks (color), jenis font (font-family), dan bayangan kotak (box-shadow) untuk elemen tertentu seperti .card 
 
Code: 

In [ ]:
body { 
    background-color: #f8f9fa; 
    font-family: Arial, sans-serif; 
}

h1 {
    color: #343a40;
}

.card {
    background-color: #ffffff;
    border: 1px solid #dee2e6;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); 
}

#output {
    background-color: #ffffff;
    color: #212529;
}

- `static/js/script.js`: File JavaScript untuk logika aplikasi. 

Analisa code :

1. Konsep Struktur Data dan Array:

- Tipe Data: Variabel apName adalah string yang berisi nilai dari elemen dengan id ap_name. Objek data merupakan sebuah objek JavaScript yang digunakan untuk mengirim data melalui fetch request.

- Struktur Data: Penggunaan objek data untuk menyimpan aksi (action) dan opsional nama titik akses (ap_name) jika aksi adalah 'show_cleartext'.

Fungsi executeAction berfokus pada mengirim permintaan POST menggunakan fetch API dari JavaScript ke endpoint /execute di server. Ini memungkinkan aplikasi web untuk berinteraksi dengan backend (misalnya, aplikasi Flask) untuk melakukan operasi berdasarkan aksi yang diberikan (seperti memeriksa host Windows, menampilkan SSID, dan sebagainya). Hasil dari operasi yang dilakukan oleh backend kemudian ditampilkan di elemen dengan id output. 

In [ ]:
function executeAction(action) { 
    let apName = document.getElementById('ap_name').value; 
    let data = { action: action }; 

    if (action === 'show_cleartext') { 
        data['ap_name'] = apName; 
    }

    fetch('/execute', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json', 
        },
        body: JSON.stringify(data), 
    })
    .then(response => response.json()) 
    .then(data => { 
        document.getElementById('output').innerText = data.result; 
    })
    .catch((error) => { 
        console.error('Error:', error); 
    });
} 

**Kesimpulan dari seluruh program *** 

Konsep Struktur Data dan Array:

- Tipe Data: File-file seperti Alat_wifi.py dan app.py kemungkinan besar mengandung definisi dan penggunaan berbagai tipe data seperti string, integer, dan mungkin beberapa struktur data seperti dictionary atau list dalam Python.

- Struktur Data: Berkaitan dengan cara data disimpan dan diorganisir di dalam file Python. Misalnya, penggunaan fungsi dan kelas untuk mengelola dan memanipulasi data